# Learning Machine Learning
Code from my experience learning the basics of machine learning from the book "Introduction to Machine Learning" by Alex Smola and SVN Vishwanathan

## Part 1: Basic Algorithms
----
### 1. Naive Bayes
Naive bayes uses bayes rule from statistics to classify a variable. An example of this used in the text is spam vs ham email. It will do this by using the various word counts of different words in each email and comparing them to the word's frequenct in spam emails vs the word's frequency in ham emails.

### Mathematics
#### Baye's Rule
The Naive bayes classifiers are based on "Bayes Rule" Which simply provides a method of calculating posterior probability from independent probabilities and the likelyhood probability. In english, it is a method of calculating the probability of an event, based on the prior knowledge of other conditions which might be related to the event. The basic equation is as follows:
$$ P( c \mid x ) = \frac{P ( x \mid c ) P( c )}{ P( x )} $$

Where : 
- $P(c \mid x)$ is the probabilty of class $c$ given predictor $x$.
- $P(x \mid c)$ is the probability of predictor $x$ given class $c$.
- $P(x)$ is the probability of predictor $x$
- $P(c)$ is the probability of class $c$

#### Baye's Rule Example
A simple example of Bayes rule in action would be this:
A Doctor gives you a blood test for the menacingly named *Scary Flu*. Let's Say the *Scary Flu* affects 1 in 1000 People in your country. As the test is being run, you read that the test is 99% accurate which means it's false positive rate is 1%. You get back a positive result. What is the probability you have the *Scary Flu*?

For this example:
- The classifier $c$ will be if you have Scary flu or not
- The predictor $x$ will be the fact you got a positive result from the test
- What we want to calculate: The probability $P( c \mid x )$, the probability that you have Scary flu given the fact you got a positive result from the test

In order to calculate this value using Bayes rule we must first calculate:
- $P(c)$ the probability of having the swine flu
$$P(c) = 1 / 1000 = .001 $$
- $P(x \mid c)$ the probability of getting a positive result on the test given you have Scary flu.
$$P(x \mid c) = 1.0 $$
- $P(x)$ the probability of getting a positive result on the test
$$P(x) = P(x \mid c)*P(c) + P(x \mid !c)*P(!c)$$
$$P(x) = 1.0*.001 + .01*.999 = 0.01099 $$ 

Now using Bayes Rule:
$$P( c \mid x ) = \frac{P(x \mid c) * P(c)}{P(x)}$$
$$P(c \mid x) = \frac{ 1.0 * .001}{0.01099} \approx 0.09$$
This means, given a positive on the test, you have around 9% chance of actually having the Scary Flu. Not so bad!

#### Naive Bayes Transition and Example
Naive Baye's classifiers use Bayes rule to predict a certain class of an object based on some predictor. This is *naive* because it assumes independence, even when that clearly is not the case. To give an example with context I will show a basic spam email filter. First, the mathematics behind it. 

The problem will be to read an email, and try to predict if that email is *spam* or *ham*. Ham being a word for an email that a person cares about. To put this into the context of Bayes rule, for any given email $x$ we want to know:
$$P(\text{ham} \mid x ) \text{ and } P( \text{spam} \mid x )$$
We know from N total training emails with $n_{\text{ham}}$ ham emails and $n_{\text{spam}}$ spam emails:
$$P(\text{ham})\approx \frac{n_{\text{ham}}}{N}$$
$$P(\text{spam})\approx \frac{n_{\text{spam}}}{N}$$
However, the other necessary quantities, specifically $P(x)$, $P(x \mid spam)$, and $P( x \mid ham )$ are unknown. We can simplify our classifier, as our problem is binary, by reducing our problem down to a Likelyhood ratio:
$$ L(x) = \frac{P(\text{spam}\mid x)}{P(\text{ham}\mid x)} $$
And using bayes rule simplify that to:
$$ L(x) = \frac{P(x\mid\text{spam})P(\text{spam})}{P(x\mid\text{ham})P(\text{ham})}$$
This effectively removes our dependency on $P(x)$ however we still need to calculate the other two quantities. 

The calculation of the quantities $P(\text{ham} \mid x)$ and $P(\text{spam} \mid x)$ are where we make our key approximation. There are many ways to do this step. In order to do this step we must decide on what metrics we will use. For this example, we will use only the probability of finding each word in a spam or ham document. The details on how we choose to do this follows.
###  1.i. Training
To show an example of this algorithm I will use [data from the University of California Irvine's machine learning databases](https://archive.ics.uci.edu/ml/index.php) which are a great place to go to get some nice datasets to learn on. The goal of the training algorithm is to get an idea of the dataset and teach an algorithm how to distinguish spam from ham. 

In [104]:
import nltk
import os
import re

# The data I am using is from https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/
labelsFile = 'SPAMTrain.label'

indir = 'ExtractedData/'
# now I build a frequency distrubtion of all the words in each type
# of email
fdistSpam = nltk.FreqDist()
fdistHam = nltk.FreqDist()

# we will train on the first 90% of our data
nTrain = 1
nTest = 1
with open(labelsFile, 'r') as flabel:
    nTotalLines = len(flabel.readlines())
    nTrain = int(nTotalLines * .9)
    nTest = int(nTotalLines * .1)
    
nSpam = 0
nHam = 0
with open(labelsFile, 'r') as flabel:
    # only go over the first nTrain lines
    # we could randomize which nTrain files to train on, for 
    # simplicity's sake I will just go over the first ones
    for _ in range(nTrain):
        line = flabel.readline()
        label = int(line.split()[0])
        fname = line.split()[1]
        with open(indir + fname, 'r', encoding="latin-1") as myfile:
            totEmail = myfile.read()
            totEmail = re.sub('<[^<]+?>', '', totEmail)
            for sent in nltk.sent_tokenize(totEmail):
                for w in nltk.word_tokenize(sent):
                    if( len(w) > 2 ):
                        if( label == 1 ): #spam
                            nSpam += 1
                            fdistSpam[w] += 1
                        else: #ham
                            nHam += 1
                            fdistHam[w] += 1

In [98]:
import numpy as np

# we can define our classify method as follows
def Classify( fEmail, c = .5 ):
    with open(fEmail, 'r', encoding="latin-1") as myfile:
        # first make a freqDist for our new file
        fdistEmail = nltk.FreqDist()
        totEmail = myfile.read()
        totEmail = re.sub('<[^<]+?>', '', totEmail)
        for sent in nltk.sent_tokenize(totEmail):
            for w in nltk.word_tokenize(sent):
                if( len(w) > 2 ):
                    fdistEmail[w] += 1
                    
        # initialize this to offset the rejection threshold
        t = -1*((np.log(c) + np.log(nHam) - np.log(nSpam)))
        offsetSpam = 1.0 / (fdistSpam.N() + len(fdistSpam))
        offsetHam = 1.0 / (fdistHam.N() + len(fdistHam))
        for w in fdistEmail.keys():
            t += fdistEmail[w] * ( np.log(fdistSpam.freq(w) + offsetSpam) - np.log(fdistHam.freq(w) + offsetHam))
        if( t > 0 ):
            return 1 #spam
        else:
            return 0 #ham
    

In [103]:
# now we test ourselves
nCorrect = 0
with open(labelsFile, 'r') as flabel:
    # skip the training data
    for _ in range(nTrain):
        flabel.readline()
    # now read the last nTest lines and classify the documents
    for _ in range(nTest):
        line = flabel.readline()
        label = int(line.split()[0])
        fname = line.split()[1]
        spam = Classify(indir + fname)
        # if we were correct in our classification, increment our 
        # correct count
        if( spam == label ):
            nCorrect += 1
    print(( 100.0 * nCorrect )/nTest ,"% Correct" )
        

96.99074074074075 % Correct


## Summary

Now our algorithm is ready to use and classify actual data simply using the classify function. The bulk of the work was done in training. classifying is lower weight as the frequency distributions have been already calculated.

Some possible ways to optimize this specific problem which would serve as good beginner exercises would be to try to pair down the dictionary to only the most polarizing words. i.e. the words that show up in spam *significantly* more than ham or in ham more than spam. This would cut down algorithm complexity on the Classifying side and would also cut down the size of the dictionaries from the training phase.

---
### 2 Nearest Neighbor Estimators
Another simple algorithm used in classifiers in machine learning is the Nearest Neighbors algorithm. It assigns the label to an observation based on the nearest neighbor(s) to that obersvation. In order to define "nearest" we need to have a distance metric. This distance is very problem specific and it allows the algorithm to be extremely flexible. For instance, in a problem involving categorizing documents we could use a distance metric of how many characters you would have to change to get from one to the other. 

In it's simplest form, this algorithm looks at only the nearest neighbor of an observation. This is prone to some serious noise, as one mislabeled element can cause a pocket around it of mislabled observations. To remedy this issue with the algorithm, it helps to take a few of the nearest neighbors and take the average label among them. This small adaption to the Nearest Neighbor algorithm ears the new name k-Nearest Neighbor. This algorithm can yeild excellent performance if the distance metric is good. This is a very popular algorithm because of it's ease of use. 

This algorithm requires very little mathematics to back it up and is most easily shown in an example. For this we will skip the mathematics section and go straight to the example.

### Example
